In [12]:
import pickle
import pandas as pd 
import numpy as np
import math
from statistics import median
import copy

In [13]:
with open('../data/trace_VecWordList.pkl',"rb") as f1, open('../data/trace_WordList.pkl', 'rb') as f2, open('../data/df_pre.pkl', 'rb') as f3:
    trace_VecWordList = pickle.load(f1) # Now holds vectors for each tokens in trace_VecWordList[trace][event][token].
    trace_WordList = pickle.load(f2) # Now holds the word tokens in same structure. Needed for idf calculation. 
    df = pickle.load(f3)

In [14]:
#trace_WordList[3][5][1]
abs(-12)

12

## Bilden eines Trace Vektors mittels IDF
In dieser Methode werden die jeweiligen Vektoren gebildet, die die gesamten Traces repräsentieren. Für eine möglichst sinnvolle Gewichtung in diesem Prozess, wird die IDF (Inverse Document Frequency) verwendet. Die IDF ist eine in der NLP verbreitete Methoden und wird häufig in Zusammenhang mit der TF (Total Frequency) verwendet. Für unseren Fall eignet sich die IDF als alleiniges Merkmal besser. Die wichtige Funktion der IDF ist es, die Wörter entsprechend der "Wichtigkeit" zu bewerten. So werden werden Wörter, welche in vielen der Event vorkommen, eine geringere IDF aufweisen, als Wörter, die selten vertreten sind (Rechtschreibfehler oder seltene Zeitkategorisierungen)

Ein weiterer wichtiger Punkt, sind die Case IDs in der Methode. Da für den Usecase der Anomaly Detection in der Regel davon auszugehen ist, dass die Case IDs eindeutig und damit nicht von den Anomalies betroffen sind, werden die jeweiligen Vectoren nicht für den später gebildeten Trace Vektor berücksichtigt. Das Hinzuziehen von weiteren Vektoren, die keinen Wert für mögliche Anomalies liefern, wäre damit lediglich Noise. Die IDF der jeweiligen Case IDs gibt allerdingds darüber Aufschluss, wie lang oder kurz eine jeweilige Trace ist. Diese Tatsache machen wir uns zu nutze und nehmen die Länge der jeweiligen Traces in unsere Berechnung mit auf.

Inwieweit die Länge einer jeweiligen Trace eine Rolle spielt ist vermutlich je nach Log unterschiedlich. Es bleibt aber zu vermuten, dass die Länge der Traces in Event Logs typischerweise normalverteilt ist. Anomalies entstehen bei dieser Verteilung daher an den äußeren Rändern. In unserer Berechnung bilden wir den Median der für die Case IDs berechneten IDF-Werte. Dieser gibt uns Aufschluss über die "typische" Länge der Traces. Hierbei wurde sicht bewusst gegen den Durschnitt entschieden, um den Einfluss möglicher Outlier gering zu halten. Später multiplizieren wir die einzelnen Vektoren der Traces noch mit der jeweiligen Abweichung vom Median, um den Abstand vom Median Wert zu verdeutlichen. Die Werte liegen hier in [1, 1.31] und haben damit erst bei großer Abweichung eine Auswirkung auf die Ergebnisse.

In [15]:
def build_traceVector(df, traceVectorList, traceWordList):
    """Method to process data created by word model. It will form a summed up vector for each trace and weigh in tokens with calculated IDF."""
    
    unique_tokens = set() # Will hold unique values for all columns. 
    
    unique_tokens.update(list(df['timestamp_diff'].map(str).unique()))
    unique_tokens.update(list(df['id'].map(str).unique()))
    unique_tokens.update(list(df['event'].str.replace('[^a-zA-Z ]', '').str.lower().str.split(' ').sum()))
    
    idf_dict = dict.fromkeys(unique_tokens, 0) # Initiate dictionary holding all unique keys with 0 as values. 
    
    #Building up dictionary and upping the counter for every event which contains the word.
    for trace in traceWordList:
        for event in trace:
            for uniqueToken in unique_tokens:
                if uniqueToken in event:
                    idf_dict[uniqueToken] += 1
    
    # Calculating IDF with log of basis 10.
    event_len = len(df)
    for key in idf_dict:
        if idf_dict[key] != 0:
            idf_dict[key] = math.log10(event_len/(idf_dict[key]+1))
    
    # Building up list of IDF values for Case IDs and calculating the median. 
    unique_IDs = df['id'].unique() # automatically sorted 
    median_list = []
    
    for caseID in unique_IDs: 
        median_list.append(idf_dict[str(caseID)])
    median_case = median(median_list) # Represents the median idf score for the case IDs.
    
    # Update traceVectorList by multiplying items with the idf score. Case ID will be multiplied by idf-score/median_case.
    idfVectorList = copy.deepcopy(traceVectorList)
    for traceID in range(len(traceVectorList)): #Representing array index and not real trace IDs
        for eventID in range(len(traceVectorList[traceID])):
            event_list = [] # List in which the multiplied vectors of each event token are stored.
            for tokenID in range(len(traceVectorList[traceID][eventID])):
                
                if tokenID != 1: # Ignore case IDs
                    idf_val = idf_dict[traceWordList[traceID][eventID][tokenID]] # Holding idf value of current token.
                    word_vec = traceVectorList[traceID][eventID][tokenID] # The vector of current token.
                    multiplied_Vec = np.multiply(idf_val,word_vec)
                    event_list.append(multiplied_Vec)
                
            idfVectorList[traceID][eventID] = event_list # Add list of vectors to new list of multiplied values.
    
    # Instead of factoring in vector of case IDs we will take the trace length into account. It is more likely to have anomalies in traced with high deviation from median length.
    # After that we will sum up the event vectors and then trace vectors in the next step.
    idfSumList = copy.deepcopy(idfVectorList) # Will hold the summed up trace vectors.
    for traceIndex in range(len(idfVectorList)):
        median_difference = abs(median_case-idf_dict[str(unique_IDs[traceIndex])])+1 # Median minus the IDF of each case ID. Works because unique_IDs and traceVectorList are sorted..
        print(f"Is multiplied by {median_difference}")
        for eventIndex in range(len(idfVectorList[traceIndex])):
            idf_EventList = np.multiply(median_difference,idfVectorList[traceIndex][eventIndex]) # Multiplying all vectos by the median difference.
            idfSumList[traceIndex][eventIndex] = np.add.reduce(idf_EventList) # Building the sum vector of the event vectors.
            
        idfSumList[traceIndex] = np.add.reduce(idfSumList[traceIndex])
            
    return idfSumList

In [16]:
returnList = build_traceVector(df, trace_VecWordList, trace_WordList)

C:\Users\nbloe\AppData\Local\Temp\ipykernel_4884\1576924413.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  unique_tokens.update(list(df['event'].str.replace('[^a-zA-Z ]', '').str.lower().str.split(' ').sum()))


Is multiplied by 1.0
Is multiplied by 1.0579919469776868
Is multiplied by 1.0
Is multiplied by 1.0579919469776868
Is multiplied by 1.0579919469776868
Is multiplied by 1.0969100130080567
Is multiplied by 1.0579919469776868
Is multiplied by 1.0579919469776868
Is multiplied by 1.2430380486862944
Is multiplied by 1.0
Is multiplied by 1.0579919469776868
Is multiplied by 1.0
Is multiplied by 1.0
Is multiplied by 1.0511525224473814
Is multiplied by 1.0
Is multiplied by 1.0511525224473814
Is multiplied by 1.0579919469776868
Is multiplied by 1.0
Is multiplied by 1.0
Is multiplied by 1.0
Is multiplied by 1.0
Is multiplied by 1.0579919469776868
Is multiplied by 1.0579919469776868
Is multiplied by 1.0511525224473814
Is multiplied by 1.3010299956639813
Is multiplied by 1.0
Is multiplied by 1.0579919469776868
Is multiplied by 1.0579919469776868
Is multiplied by 1.0
Is multiplied by 1.0579919469776868
Is multiplied by 1.0579919469776868
Is multiplied by 1.0
Is multiplied by 1.0
Is multiplied by 1.057

In [17]:
returnList[0]

array([ 2.7310238 , -2.1717832 ,  3.5912461 , -0.1418205 , -5.090647  ,
       -5.203024  , -0.98290503,  6.5118775 , -3.3807192 ,  0.87190175,
       -1.1689278 , -1.6422386 , -0.4415833 ,  2.982179  , -0.49915227,
        1.070962  , -4.81453   ,  1.2765594 , -2.738927  ,  0.43211514,
       -1.5205444 ,  2.9263706 ,  5.636519  , -3.5867572 , -1.5609154 ,
        1.0828643 ,  3.008002  , -0.31009275, -6.846821  ,  1.3191539 ,
        1.4164375 ,  0.44646877, -1.7635384 ,  0.14491937,  4.842037  ,
        0.6883092 ,  5.1002736 , -1.5883536 ,  0.09477   , -0.39996237,
       -1.5763885 ,  2.4462097 , -3.5952961 , -1.2698674 ,  0.5455188 ,
        0.31671986,  1.6351323 ,  1.8557162 ,  3.501288  , -2.9405594 ,
       -3.7300618 ,  1.4265077 , -1.4943196 , -2.29262   ,  6.874989  ,
       -2.8712564 ,  3.0743315 ,  0.3647246 , -2.7540174 , -4.749709  ,
        1.254095  , -1.9753501 ,  0.73861814, -0.47637576,  2.020488  ,
        3.9870534 , -0.84499925,  2.573475  , -1.4185975 , -0.71